In [1]:
3+2

5

In [2]:
import xlwings

In [3]:
xlwings.sheet

AttributeError: module 'xlwings' has no attribute 'sheet'

In [4]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [5]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [6]:
xlwings.Range("A1").expand

<bound method Range.expand of <Range [통합 문서1]Sheet1!$A$1>>

In [7]:
xlwings.Range("A1").expand.value

AttributeError: 'function' object has no attribute 'value'

In [9]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그#4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그#6', 'chrome'],
 ['로그#7', 'chrome'],
 ['로그#8', 'chrome'],
 ['로그#9', 'chrome']]

In [10]:
column_names = total_rows[0]
column_names

['message', 'browser']

In [12]:
for row in total_rows[1:]:
    row

In [13]:
for row in total_rows[1:]:
    row_dict = dict(zip(column_names, row))
    print(row_dict)

{'message': '첫 로그', 'browser': 'chrome'}
{'message': '두번째 로그', 'browser': 'chrome'}
{'message': '세번째 로그', 'browser': 'chrome'}
{'message': '로그#4', 'browser': 'chrome'}
{'message': '로그#5', 'browser': 'chrome'}
{'message': '로그#6', 'browser': 'chrome'}
{'message': '로그#7', 'browser': 'chrome'}
{'message': '로그#8', 'browser': 'chrome'}
{'message': '로그#9', 'browser': 'chrome'}


In [17]:
rows_data = [dict(zip(column_names, row))
    for row in total_rows[1:]]
                  
rows_data

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

In [18]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

In [ ]:
DATABASE = {
    "host" : "127.0.0.1", 
    "user" : "root", 
    "password" : "1234", 
    "db" : "myproj11_db", 
    "charset" : "utf8",
}

In [25]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log")
print(cursor.fetchall())
cursor.close()

()


In [26]:
# with절 적용

In [27]:
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [28]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    print(cursor.fetchall())

()


# header

In [35]:
column_data = {'message': '첫 로그', 'browser': 'chrome'}

with db_cursor() as cursor:
    for row_dict in get_rows_data():
        sql = "INSERT INTO LOG(message, browser) VALUES(%(message)s, %(browser)s)"
        cursor.execute(sql, ["첫 로그", "chrome"])
        cursor.commit()

OperationalError: (1364, "Field 'id' doesn't have a default value")

In [29]:
get_rows_data()

[{'message': '첫 로그', 'browser': 'chrome'},
 {'message': '두번째 로그', 'browser': 'chrome'},
 {'message': '세번째 로그', 'browser': 'chrome'},
 {'message': '로그#4', 'browser': 'chrome'},
 {'message': '로그#5', 'browser': 'chrome'},
 {'message': '로그#6', 'browser': 'chrome'},
 {'message': '로그#7', 'browser': 'chrome'},
 {'message': '로그#8', 'browser': 'chrome'},
 {'message': '로그#9', 'browser': 'chrome'}]

In [41]:
with db_cursor() as cursor:
    sql = "SELECT * FROM LOG WHERE id = 1"
    cursor.execute(sql)
    for row in cursor:
        print(row)

(1, '첫 로그', 'chrome')


In [42]:
with db_cursor() as cursor:
    log_id = 1
    sql = "SELECT * FROM LOG WHERE id = 1"
    cursor.execute(sql)
    for row in cursor:
        print(row)

(1, '첫 로그', 'chrome')


In [43]:
with db_cursor() as cursor:
    log_id = 1
    sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
    cursor.execute(sql)
    for row in cursor:
        print(row)

(1, '첫 로그', 'chrome')


In [53]:
with db_cursor() as cursor:
    log_id = '1 or 1'
    #sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
    #sql = "SELECT * FROM LOG WHERE id = 1 or 1 "
    
    sql = "SELECT * FROM LOG WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

(1, '첫 로그', 'chrome')


In [55]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#    for row in cursor:
#        row[0] #id
#        row[1] #message
#        row[2] #chrome
#        print(row)

rows_data

((1, '첫 로그', 'chrome'), (2, '두번째', 'chrome'), (3, '세번째', 'chrome'))

In [79]:
from collections import namedtuple

Log = namedtuple('Log', 'id message browser')

In [80]:
log = Log(1, 'message' ,'chrome')
print(row[0])
print(row[1])
print(row[2])

3
세번째
chrome


In [56]:
rows_data

((1, '첫 로그', 'chrome'), (2, '두번째', 'chrome'), (3, '세번째', 'chrome'))

In [63]:
Log = log(1, 'message' ,'chrome')

NameError: name 'log' is not defined

In [64]:
log = Log(1, 'message' ,'chrome')

TypeError: 'tuple' object is not callable

In [83]:
log.id, log.message, log.browser

(1, 'message', 'chrome')

In [82]:
log = Log(1, 'message' ,'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [81]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()]
rows_data

[Log(id=1, message='첫 로그', browser='chrome'),
 Log(id=2, message='두번째', browser='chrome'),
 Log(id=3, message='세번째', browser='chrome')]